# 01 时间动力学与异常检测

目标：构建日粒度时间序列，执行 STL 分解、突发检测、异常推文标记。

In [3]:
import sys
from pathlib import Path

# 将项目根目录添加到 Python 路径
project_root = Path('/workspace')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
print(f"✅ Python 路径已配置: {project_root}")

✅ Python 路径已配置: /workspace


## 步骤 1: 加载已处理的数据

In [ ]:
from src import analysis
import polars as pl
from pathlib import Path

# 加载 parquet 数据
df = pl.read_parquet("../parquet/tweets_enriched.parquet")
print(f"📊 数据加载完成: {df.height:,} 行, {df.width} 列")
print(f"\n数据时间范围:")
print(f"  最早: {df['createdAt'].min()}")
print(f"  最晚: {df['createdAt'].max()}")

## 步骤 2: 构建时间序列

In [ ]:
# 创建互动指标列（如果不存在）
if 'total_engagement' not in df.columns:
    df = df.with_columns(
        (pl.col('retweetCount') + pl.col('replyCount') + pl.col('likeCount') + pl.col('quoteCount')).alias('total_engagement')
    )

# 使用 analysis 模块构建时间序列
ts_profile = analysis.build_time_series(df, 'createdAt', 'total_engagement')

print(f"📈 时间序列构建完成:")
print(f"  每日统计: {ts_profile.daily_counts.height} 天")
print(f"  滚动指标: {ts_profile.rolling_metrics.height} 条记录")
print(f"  异常天数: {ts_profile.anomalies.height} 天")

print(f"\n异常日期示例:")
print(ts_profile.anomalies.head(5))

## 步骤 3: 保存时间序列数据

In [ ]:
from src import io

# 保存每日统计
daily_path = Path("../parquet/tweets_daily.parquet")
io.materialize_parquet(ts_profile.daily_counts.lazy(), daily_path)
print(f"✅ 每日统计已保存: {daily_path}")

# 保存滚动指标
rolling_path = Path("../parquet/tweets_rolling.parquet")
io.materialize_parquet(ts_profile.rolling_metrics.lazy(), rolling_path)
print(f"✅ 滚动指标已保存: {rolling_path}")

# 保存异常检测结果
anomaly_path = Path("../parquet/tweets_anomalies.parquet")
io.materialize_parquet(ts_profile.anomalies.lazy(), anomaly_path)
print(f"✅ 异常检测结果已保存: {anomaly_path}")

print(f"\n📂 生成的文件:")
for f in io.list_parquet_files():
    print(f"  - {f}")

## ✅ 时间序列分析完成！

接下来可以运行 `02_network_intelligence.ipynb` 进行网络分析。